In [0]:
%sql
USE CATALOG lab;
USE SCHEMA data;

# Exploratory Data Analysis

In [0]:
%sql
-- Select all columns to review channel-level KPIs
SELECT *
FROM lab.data.channel_daily_gold
ORDER BY date DESC, spend_usd DESC; -- Order by most recent date, then by channel

In [0]:
%sql
-- Completeness checks on key columns
SELECT
  COUNT(*) AS total_rows,
  COUNT(date) AS non_null_dates,
  COUNT(revenue_usd) AS non_null_revenue,
  COUNT(spend_usd) AS non_null_spend
FROM channel_daily_gold;

In [0]:
%sql
-- Summary by channel
SELECT
  channel,
  COUNT(*) AS days,
  SUM(spend_usd) AS spend_usd,
  SUM(revenue_usd) AS revenue_usd,
  AVG(ctr) AS avg_ctr,
  AVG(roas) AS avg_roas
FROM channel_daily_gold
GROUP BY channel
ORDER BY revenue_usd DESC;

In [0]:
%sql
-- Create a daily total KPI view across all channels
With vw_daily_total_kpis AS
(SELECT
  date,
  SUM(spend_usd) AS spend_usd,
  SUM(impressions) AS impressions,
  SUM(clicks) AS clicks,
  CASE WHEN SUM(impressions) > 0 THEN SUM(clicks) * 1.0 / SUM(impressions) ELSE 0 END AS ctr,
  SUM(revenue_usd) AS revenue_usd
FROM channel_daily_gold
GROUP BY date),

-- 7-day rolling average
rev_trend as
(SELECT
  date,
  revenue_usd,
  AVG(revenue_usd) OVER (
    ORDER BY date
    ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
  ) AS revenue_usd_ma7
FROM vw_daily_total_kpis
ORDER BY date),

-- Weekday seasonality (across whole series)
seasonality as
(SELECT
  DAYOFWEEK(date) AS dow,
  DATE_FORMAT(date, 'E') AS weekday,
  AVG(revenue_usd) AS avg_revenue,
  AVG(spend_usd) AS avg_spend,
  AVG(clicks) AS avg_clicks
FROM vw_daily_total_kpis
GROUP BY 1,2
ORDER BY 1),

corr as (
  -- Correlation between spend and revenue (total series)
SELECT corr(spend_usd, revenue_usd) AS corr_spend_revenue
FROM vw_daily_total_kpis
)

-- select * from corr
-- select * from seasonality
-- select * from rev_trend
select * from vw_daily_total_kpis

In [0]:
%sql
-- Channel share over the last 7 days
WITH recent AS (
  SELECT *
  FROM lab.data.channel_daily_gold
  WHERE date >= DATEADD(day, -7, CURRENT_DATE())
)
SELECT
  channel,
  SUM(revenue_usd) AS revenue_usd,
  SUM(spend_usd) AS spend_usd,
  SUM(clicks) AS clicks
FROM recent
GROUP BY channel
ORDER BY revenue_usd DESC;